In [119]:
import pandas as pd
import spacy

In [120]:
file_path = r"data\Goethe Wortliste A2.csv"
nlp = spacy.load("de_core_news_lg")

df = pd.read_csv(file_path, header=0, delimiter="|")
df = df.drop("Audio_Wort", axis=1)
df.head()

,Word_DE,Word_EN,Plural,Hinweis,Verbformen,S1_DE,S1_EN,S2_DE,S2_EN,S3_DE,S3_EN,S4_DE,S4_EN
0,anbieten,to offer,NaN,NaN,"bietet an, hat angeboten",Darf ich Ihnen ein Stück Kuchen anbieten?,May I offer you a piece of cake?,Er hat mir eine Stelle als Verkäuferin angeboten.,He offered me a job as a saleswoman.,NaN,NaN,NaN,NaN
1,abgeben,"to give (sth.) in, to submit (sth.), to delive...",NaN,NaN,"gibt ab, hat abgegeben",Ich muss meinen Schlüssel an der Rezeption abg...,I have to leave my key at the front desk.,NaN,NaN,NaN,NaN,NaN,NaN
2,anders,different,NaN,NaN,NaN,Anders geht das leider nicht.,It does not go otherwise.,Oliver ist anders als seine Freunde.,Oliver is different from his friends.,NaN,NaN,NaN,NaN
3,abschließen,"to lock, to finish",NaN,NaN,"schließt ab, hat abgeschlossen",Hast du die Tür abgeschlossen?,Did you close the door?,Ich schließe dieses Jahr meine Ausbildung ab.,I finish my education this year.,Ich schließe dieses Jahr mein Studium ab.,I finish my studies this year.,NaN,NaN
4,die Adresse,address,-n,NaN,NaN,Können Sie mir Ihre Adresse geben?,Can you give me your address?,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
df1 = df[["S1_DE"]].rename(columns={"S1_DE": "sentence"})
df2 = df[["S2_DE"]].rename(columns={"S2_DE": "sentence"})
df3 = df[["S3_DE"]].rename(columns={"S3_DE": "sentence"})
df4 = df[["S4_DE"]].rename(columns={"S4_DE": "sentence"})
df_s = pd.concat([df1, df2, df3, df4], ignore_index=True).dropna().drop_duplicates().reset_index().drop("index", axis=1)
df_s.tail()

,sentence
1173,Haben Sie die Bluse auch eine Nummer kleiner?
1174,Ich finde keinen Parkplatz.
1175,Wann kommst du wieder?
1176,Dies ist besonders beim ersten Wechsel leider ...
1177,"Aber sie zu dritt zu überreden, ist eine gute ..."


In [122]:
df_w = pd.DataFrame({"word":[], "lemma":[], "pos":[]})
df_w.head()
for i in range(0, len(df_s)):
    for token in nlp(df_s.loc[i, "sentence"]):
        # print(f'{token.text:<12} => {token.lemma_:<12}')
        df_w.loc[len(df_w)] = [token.text, token.lemma_, token.pos_]
    # print("\n")

In [123]:
df_1 = df_w.groupby("word").agg(freq=("word", "count")).reset_index().sort_values("freq", ascending=False)
# df_w.head(50).sort_values("freq", ascending=False)

In [124]:
df_w = pd.merge(df_1, df_w, on="word").drop_duplicates().reset_index().drop("index", axis=1)
# Remove all punctuation
df_w.drop(df_w[df_w["pos"]=="PUNCT"].index, inplace=True)
df_w = df_w.reset_index().drop("index", axis=1)
df_w.tail()

,word,freq,lemma,pos
2297,Wettbewerb,1,Wettbewerb,NOUN
2298,Wen,1,wen,PRON
2299,Welchen,1,welchenr,DET
2300,Erzählst,1,erzählst,ADV
2301,Witz,1,Witz,NOUN


## Cleaning Verbs

In [494]:
verbs = ["sein", "haben", "werden", "sagen", "geben", "kommen", "wissen", "machen", "denken", "nehmen", "sehen", "finden", "gehen", "stehen", "liegen", "heißen", "fahren", "bleiben", "lassen", "sprechen", "bringen", "leben", "schreiben", "laufen", "beginnen", "trinken", "essen", "spielen", "arbeiten", "brauchen", "sich", "hören", "kaufen", "tragen", "verstehen", "lesen", "fliegen", "fallen", "singen", "schlafen", "zahlen", "sich", "suchen", "warten", "sich", "anfangen", "erzählen", "wohnen", "fragen", "helfen", "müssen", "können", "mögen", "dürfen", "tun", "bekommen", "vergessen", "schmecken", "wollen", "dauern", "treffen", "bitten", "gefallen", "verlieren", "überreden", "freuen", "passen", "teilen", "regnen", "fehlen", "erreichen", "schwimmen", "ziehen", "erlauben", "verlassen", "bedeuten", "stellen", "benutzen", "beeilen", "bedienen", "kehren", "kennen", "passieren", "besuchen", "interessieren", "waschen", "bestellen", "sollen", "aussteigen", "ausziehen", "besichtigen", "verabreden", "unterstützen", "unterrichten", "umschauen", "anmelden", "kosten", "absagen", "buchen", "wählen", "sollen", "drücken", "zutreffen", "öffnen", "schicken", "schneiden", "schließen", "meinen", "schneien", "schenken", "schlagen", "mitnehmen", "reisen", "organisieren", "parken", "rufen", "kriegen", "danken", "halten", "feiern", "fühlen", "gewinnen", "einsteigen", "empfehlen", "entscheiden", "enttäuschen", "erkälten", "erledigen", "glauben", "gießen", "kümmern", "lernen", "lügen", "besetzen", "bestehen", "betreuen", "beweisen", "anziehen", "kontrollieren", "anstoßen", "kochen", "begleiten", "unterhalten", "unterschreiben", "wegwerfen", "wechseln", "streiten", "wiederholen", "versuchen", "verschieben", "vertun", "verwenden", "verneinen", "vorbereiten", "verletzen", "überweisen", "sitzen", "übernachten", "schätzen", "verteilen", "weitermachen", "weiterhelfen", "klappen", "ignorieren", "weinen", "wandern", "wegfahren", "joggen", "wecken", "wehtun", "informieren", "wegmachen", "wegnehmen", "schaffen", "geschehen", "speichern", "sterben", "träumen", "ändern", "zusammenleben", "gelten", "zurückgeben", "übersetzen", "gehören", "überqueren", "kündigen", "lachen", "lieben", "üben", "pflanzen", "ärgern", "gratulieren", "wiegen", "werben", "hoffen", "holen", "husten", "hängen", "grillen", "zeigen", "zeichnen", "raten", "putzen", "prüfen", "prägen", "pausieren", "rauchen", "spazieren", "reduzieren", "reden", "sparen", "rausbringen", "rechnen", "rauskommen", "studieren", "stören", "notieren", "stimmen", "reinkommen", "schimpfen", "scheinen", "schauen", "saugen", "riechen", "reparieren", "renovieren", "reiten", "sammeln", "räumen", "leiten", "verbinden", "verbieten", "leihen", "verbreiten", "verdienen", "vereinbaren", "verfeinern", "verbessern", "unternehmen", "untersuchen", "lauten", "verfügen", "verreisen", "verstecken", "vermieten", "verpassen", "vergleichen", "verhalten", "verkaufen", "verlieben", "liefern", "treiben", "mitmachen", "mitbringen", "trainieren", "teilnehmen", "tauschen", "mieten", "merken", "laden", "malen", "umsteigen", "umziehen", "notieren", "füllen", "legen", "konzentrieren", "chatten", "braten", "blicken", "aufräumen", "beheben", "begründen", "steigen", "ankommen", "beenden", "beantworten", "backen", "austauschen", "ausmachen", "ausgehen", "ausgeben", "ausfüllen", "anbieten", "bieten", "beobachten", "biegen", "bezahlen", "bewerben", "bestätigen", "beschäftigen", "beschweren", "berichten", "drucken", "freilassen", "fotografieren", "folgen", "fernsehen", "gebären", "bringen", "basteln", "fürchten", "führen", "funktionieren", "frühstücken", "einladen", "einkaufen", "einpacken", "erlernen", "erwachen", "erleben", "eintragen", "entwickeln", "einziehen", "aufmachen", "aufhören", "stören", "abholen", "abschließen", "abgeben", "abbiegen", "dürfen", "ansehen", "anrufen", "ansetzen", "erkennen", "erlöschen", "setzen", "werfen", "erklären", ]

In [258]:
# update "müssen" lemma
mask = (df_w["word"].str.contains("^m[uü]ss", regex=True, case=False)) & (df_w["pos"]=="AUX")
df_w.loc[mask, "lemma"] = "müssen"

# update "haben" lemma
mask = (df_w['word'].str.contains("^ha[sbt][et]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "haben"

# update "mögen" lemma
mask = (df_w['word'].str.contains("möchte", case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "mögen"

# update "können" lemma
mask = (df_w['word'].str.contains("^(?:gek|k)[aoö]nn[se]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "können"

# update "gehen" lemma
mask = (df_w['word'].str.contains("^(?:gegang|geh|ging)[se]?[tn]?$", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "gehen"

# update "bitten" lemma
mask = (df_w['word'].str.contains("^bitte[s]?[tn]?|^gebeten|^b[aä]t[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "bitten"

# update "gefallen" lemma
mask = (df_w['word'].str.contains("^gefäll", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "gefallen"

# update "machen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)mach[t]?[es]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "machen"

# update "wollen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)w[io]ll[t]?[se]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "wollen"

# update "kommen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)k[oaä]m[m]?[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "kommen"

# update "bekommen" lemma
mask = (df_w['word'].str.contains("^bek[oaä]m[m]?[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "bekommen"

# update "ziehen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)z(?:[oö]g|ieh)[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "ziehen"

# update "ausziehen" lemma
mask = (df_w['word'].str.contains("^aus(?:ge|)z(?:[oö]g|ieh)[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "ausziehen"

# update "fahren" lemma
mask = (df_w['word'].str.contains("^(?:ge|)f[aäu]hr[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "fahren"

# update "essen" lemma
mask = (df_w['word'].str.contains("^(?:gege|)[eia](?:ss|ß)[e]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "essen"

# update "finden" lemma
mask = (df_w['word'].str.contains("^(?:ge|)f[iu]nd[e]?(?:st|t|n)", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "finden"

# update "sein" lemma
mask = (df_w['word'].str.contains("^bi(?:n|st)|^sind|^sei(?:d|n|en|st|est|et|)|^gewesen|^w[aä]r(?:st|t|en)$", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "sein"

# update "meinen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)mein[t]?[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "meinen"

# update "vergessen" lemma
mask = (df_w['word'].str.contains("^verg(?:[ie]ss|[aä]ß)[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "vergessen"

# update "stellen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)stell[t]?[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "stellen"

# update "brauchen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)brauch[t]?[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "brauchen"

# update "kennen" lemma
mask = (df_w['word'].str.contains("^(?:kenn[t]?[e]?[s]?[tn]?|(?:ge|)kannt[e]?[s]?[tn]?)", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "kennen"

# update "hören" lemma
mask = (df_w['word'].str.contains("^hör[t]?[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "hören"

# update "öffnen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)öffn(?:et|)[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "öffnen"

# update "wissen" lemma
mask = (df_w['word'].str.contains("^(?:weiß[t]?|wiss[e]?[tn]?|(?:ge|)w[uü]sst[e]?[s]?[tn]?)", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "wissen"

# update "schneiden" lemma
mask = (df_w['word'].str.contains("^(?:ge|)schn(?:eid|itt)[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "schneiden"

# update "geben" lemma
mask = (df_w['word'].str.contains("^(?:(?:ge|)g[eiaä]b[e]?[s]?[tn]?|gibt's)$", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "geben"

# update "gießen" lemma
mask = (df_w['word'].str.contains("^(?:(?:ge|)g[oö]ss|gie(?:ss|ß))[e]?[s]?[tn]?$", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "gießen"

# update "sprechen" lemma
mask = (df_w['word'].str.contains("^(?:gesprochen|spr[eiaä]ch[e]?[s]?[tn]?)", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "sprechen"

# update "steigen" lemma
mask = (df_w['word'].str.contains("^(?:(?:ge|)stieg|steig)[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "steigen"

# update "schauen" lemma
mask = (df_w['word'].str.contains("^(?:ge|)schau[t]?[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "schauen"

# update "lassen" lemma
mask = (df_w['word'].str.contains("^(?:(?:ge|)l[aä]ss|ließ)[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "lassen"

# update "verbieten" lemma
mask = (df_w['word'].str.contains("^verb(?:ie|[oö])t[e]?[s]?[tn]?", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "verbieten"

# update "nehmen" lemma
mask = (df_w['word'].str.contains("^(?:n[eaä]hm[e]?[s]?[tn]?|genommen|nimm[s]?[t]?)", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "nehmen"


In [480]:
# update "nehmen" lemma
mask = (df_w['word'].str.contains("^(?:n[eaä]hm[e]?[s]?[tn]?|genommen|nimm[s]?[t]?)", regex=True, case=False)) & (df_w["pos"].isin(["AUX", "VERB"]))
df_w.loc[mask, "lemma"] = "nehmen"

df_w.loc[mask].head(50)

,word,freq,lemma,pos
273,nehme,6,nehmen,VERB
727,nehmen,2,nehmen,VERB
999,Nehmen,2,nehmen,VERB
1206,genommen,1,nehmen,VERB
1640,Nimm,1,nehmen,VERB
2398,nimmst,1,nehmen,VERB
2399,nahmst,1,nehmen,VERB
2400,nehmt,1,nehmen,VERB


In [189]:
def add_new_word(df, word, lemma, pos) :
    new_word = {
        "word": word,
        "freq": 1,
        "lemma": lemma,
        "pos": pos
    }
    
    df.loc[len(df)] = new_word
    return  df

In [478]:
# df_w = add_new_word(df_w, "nehmt", "nehmen", "VERB")

In [499]:
mask = df_w["pos"].isin(["AUX", "VERB"])
print("Total unfinished words:", len(df_w.loc[mask]))
print(len(df_w.loc[mask]))
print(len(df_w.loc[mask].drop_duplicates(subset=["word"])))

Total unfinished words: 698
698
675


In [492]:
# df_w.loc[2248, "lemma"] = "anfangen"

# mask = (df_w['word'].str.contains("[t]?(?:e|st|t|en)$", regex=True, case=False)) &\
#     (df_w["pos"].isin(["AUX", "VERB"])) &\
#     ~(df_w["lemma"].isin(verbs))

# df_w.loc[mask].head(50).drop_duplicates(subset=["lemma"])

## Cleaning Pronouns

In [540]:
pronouns = ["ich", "du", "er", "sie", "es", "wir", "ihr", "Sie", "mein", "dein", "sein", "ihr", "unser", "euer", "mich", "dich", "sich", "uns", "euch", "sich", "der", "das", "die", "dieser", "dieses", "diese", "welcher", "welches", "welche", "jemand", "niemand", "etwas", "nichts", "jede", "einige", "mache", "kein", "alle", "viele", "man", "was", "wer", "wen", "wem", "wessen", "mir", "dir", "sich", "uns", "euch", "sich", "Ihnen", "andere", "erste"]
pos = ["PRON"]
mask = (df_w.pos.isin(pos)) & ~(df_w.lemma.isin(pronouns))
print("Total unfinished words:", len(df_w.loc[mask]))
df_w.loc[mask]

Total unfinished words: 3


,word,freq,lemma,pos
138,Ihnen,12,ihnen,PRON
542,ihm,3,ihm,PRON
1762,Ihm,1,ihm,PRON


In [553]:
# nt_indexes = [138, 542, 1762]

# mask_to_file = (df_w.pos.isin(pos))
# file_name_new = r"data\clean_pron.csv"
# df_w.loc[mask_to_file].drop(index=nt_indexes).drop("freq", axis=1).set_index("lemma").drop_duplicates().sort_index().to_csv(file_name_new)

## Cleaning Next

In [561]:
df_w.pos.unique()
# ['PRON', 'AUX', 'DET', 'PART', 'ADP', 'ADV', 'VERB', 'CCONJ', 'SCONJ', 'NOUN', 'X', 'ADJ', 'NUM', 'PROPN', 'INTJ']
pos = ["NOUN"]
mask = mask = (df_w.pos.isin(pos))
df_w.loc[mask]

,word,freq,lemma,pos
82,Bitte,21,bitte,NOUN
86,Uhr,21,Uhr,NOUN
113,Kinder,15,Kind,NOUN
120,Arbeit,14,Arbeit,NOUN
122,Auto,14,Auto,NOUN
...,...,...,...,...
2294,Wiederhören,1,Wiederhören,NOUN
2295,Ermäßigung,1,Ermäßigung,NOUN
2296,Ernstes,1,Ernst,NOUN
2297,Wettbewerb,1,Wettbewerb,NOUN
